<a href="https://colab.research.google.com/github/e184633/nlu_paper/blob/main/Reproduce_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation of a QA System

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial5_Evaluation.ipynb)

To be able to make a statement about the performance of a question-answering system, it is important to evalute it. Furthermore, evaluation allows to determine which parts of the system can be improved.

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install datasets
# If you run this notebook on Google Colab, you might need to
# restart the runtime after installing haystack.
!pip install "ray[default]"

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-i3ps0ls4
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-i3ps0ls4
  Using cached huggingface_hub-0.0.8-py3-none-any.whl (34 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.12.1 requires huggingface-hub<0.1.0,>=0.0.14, but you have huggingface-hub 0.0.8 which is incompatible.
  Using cached huggingface_hub-0.0.19-py3-none-any.whl (56 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.8
    Uninstalling huggingface-hub-0.0.8:
      Successfully uninstalled huggingface-hub-0.0.8
ERROR: pip's depende

In [ ]:
from haystack.modeling.utils import initialize_device_settings

device, n_gpu = initialize_device_settings(use_cuda=True)

In [ ]:

!nvidia-smi

Wed Oct 13 07:28:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    31W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import logging
logging.set_verbosity_error()

In [ ]:

from google.colab import files

# sas_squad.sort_values('sas').to_csv('squad_sas_score.csv') 
# files.download('squad_sas_score.csv')

In [ ]:
# url = 'https://github.com/rajpurkar/SQuAD-explorer/blob/master/dataset/dev-v2.0.json'


## Bi-encoder

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/541 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/188 [00:00<?, ?B/s]

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/T-Systems-onsite_cross-en-de-roberta-sentence-transformer. Creating a new one with MEAN pooling.


In [ ]:
!ls data/data/

GermanQuAD_SAS.csv  NQ-open_SAS.csv  README.md	SQuAD_SAS.csv


## Data Import

In [ ]:
from haystack.preprocessor.utils import fetch_archive_from_http

# Download evaluation data, which is a subset of 
# Natural Questions development set containing 50 documents
doc_dir = "data"
s3_url = " https://semantic-answer-similarity.s3.amazonaws.com/data.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

False

In [ ]:
import pandas as pd
squad = pd.read_csv('data/data/SQuAD_SAS.csv')
nq_open = pd.read_csv('data/data/NQ-open_SAS.csv')
german_quad = pd.read_csv('data/data/GermanQuAD_SAS.csv')

In [ ]:
dataset_to_bi_encoder_similarity_scores = {}
for data, name in zip([squad, nq_open, german_squad], 
                      ('squad', 'nq-open', 'german_squad')):
    scores_bi_encoder = data.copy()
    from sentence_transformers import util
    #Compute embedding for both lists
    embeddings1 = model.encode(data['answer1'], convert_to_tensor=True)
    embeddings2 = model.encode(data['answer2'], convert_to_tensor=True)

    #Compute cosine-similarits
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

    #Output the pairs with their score
    scores_bi_encoder['bi_encoder'] = pd.Series()
    for i in range(len(data['answer1'])):
        scores_bi_encoder['bi_encoder'][i] = cosine_scores[i][i].item()
    scores_bi_encoder.sort_values('bi_encoder').to_csv(f'{name}_bi_encoder_score.csv') 
    files.download(f'{name}_bi_encoder_score.csv')
    dataset_to_bi_encoder_similarity_scores[name] = scores_bi_encoder

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

compute_f1(str(4), str(5))

0

In [ ]:


from haystack.modeling.evaluation.squad_evaluation import compute_f1

In [ ]:

dataset_to_bi_encoder_similarity_scores = {}
for data, name in zip([squad, nq_open, german_squad], 
                      ('squad', 'nq-open', 'german_squad')):
    scores_bi_encoder = data.copy()
    from sentence_transformers import util
    #Compute embedding for both lists
    embeddings1 = model.encode(data['answer1'], convert_to_tensor=True)
    embeddings2 = model.encode(data['answer2'], convert_to_tensor=True)

    #Compute cosine-similarits
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

    #Output the pairs with their score
    scores_bi_encoder['bi_encoder'] = pd.Series()
    scores_bi_encoder['f1'] = pd.Series()
    for i in range(len(data['answer1'])):
        scores_bi_encoder['bi_encoder'][i] = cosine_scores[i][i].item()
    
    scores_bi_encoder['f1'] = scores_bi_encoder.apply(
        lambda x: compute_f1(str(x.answer1), str(x.answer2)), axis=1)
    scores_bi_encoder.sort_values('bi_encoder').to_csv(f'{name}_bi_encoder_score.csv') 
    files.download(f'{name}_bi_encoder_score.csv')
    dataset_to_bi_encoder_similarity_scores[name] = scores_bi_encoder


Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# temp['f1'] = temp.apply(lambda x: compute_f1(str(x.answer1), str(x.answer2)), axis=1)

In [ ]:

from transformers import AutoConfig
model_names = ['sentence-transformers/paraphrase-multilingual-mpnet-base-v2', 
               'cross-encoder/stsb-roberta-large', 
               'deepset/gbert-base',
               'deepset/gbert-large-sts',
               'T-Systems-onsite/cross-en-de-roberta-sentence-transformer'
              ]
configs = [AutoConfig.from_pretrained(model_name) for model_name in model_names]
[arch.endswith('ForSequenceClassification') for config in configs for arch in config.architectures]

Downloading:   0%|          | 0.00/728 [00:00<?, ?B/s]

[False, True, False, True, False]

## Cross-encoder

In [ ]:
from sentence_transformers import CrossEncoder
model_cross_encoder = CrossEncoder('deepset/gbert-large-sts')


Downloading:   0%|          | 0.00/728 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/240k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

In [ ]:

cross_encoder_sas = squad.copy()

In [ ]:

series = _

In [ ]:

sas_squad = squad.copy()

In [ ]:

sas_squad['sas'] = series

In [ ]:

sas_squad.sort_values('sas')

,answer1,answer2,label,sas
630,Mao Zedong,Sultan Galiev and Vasyl Shakhrai,0.0,0.001090
227,all other animals,Bilateria,0.0,0.004672
540,Lake Constance,West by the Alter Rhein,0.0,0.009475
85,William the Silent,Pierre L'Oyseleur,0.0,0.012882
619,the British Empire,Terra nullius,0.0,0.021044
...,...,...,...,...
130,Liquid oxygen,liquid oxygen,2.0,0.977769
14,Complexity measures,complexity measures,2.0,0.980499
15,Complexity measures,complexity measures,2.0,0.980499
793,there is no known polynomial-time solution,no known polynomial-time solution,2.0,0.994736


In [ ]:
from google.colab import files

sas_squad.sort_values('sas').to_csv('squad_sas_score.csv') 
files.download('squad_sas_score.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import f1_score

In [ ]:

output = _

In [ ]:

series_german = pd.Series(
    [model_cross_encoder.predict([str(german_squad.answer1.values[i]), 
                                  str(german_squad.answer2.values[i])], 
                                 show_progress_bar=False) 
                                 for i in range(len(german_squad))])

In [ ]:
nq_open['f1'] = nq_open.apply(lambda x: compute_f1(x.answer1, x.answer2), axis=1)

In [ ]:

german_quad = german_squad.copy()
german_quad['sas'] = series_german

german_quad.sort_values('sas').to_csv('german_quad_sas.csv') 
files.download('german_quad_sas.csv')

In [ ]:


series_nq_open = pd.Series(
    [model_cross_encoder.predict([str(nq_open.answer1.values[i]), 
                                  str(nq_open.answer2.values[i])], 
                                 show_progress_bar=False) 
                                 for i in range(len(nq_open))])

nq_open_sas = nq_open.copy()
nq_open_sas['sas'] = series_nq_open
nq_open_sas.sort_values('sas').to_csv('nq_open_sas.csv') 
files.download('nq_open_sas.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# series_german_quad = pd.Series(
#     [model_cross_encoder.predict([german_squad.answer1.values[i], 
#                                   german_squad.answer2.values[i]]) 
#                                  for i in range(len(german_squad))])

In [ ]:
model_cross_encoder.predict(["sentence one.","sentence two."]) 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.60535014

In [ ]:
model_cross_encoder.predict()

array(['rotiert', 'in Blautönen gehaltenen Außenwände'], dtype=object)

In [ ]:

data.answer1.values[5:6]

array(['denn es ist eine beliebte Praxis, verteilt auf die dreißig Ramadan-Nächte, eine Chatma, also eine Komplettlesung des Korans, vorzunehmen'],
      dtype=object)

In [ ]:

data.answer2.values[5:6]

array(['Sie finden vor allem in der Liturgie Verwendung und dienen als Maßeinheiten zur Festlegung von Gebetspensen'],
      dtype=object)

In [ ]:
model_cross_encoder.predict([data.answer1.values[5:6], data.answer2.values[5:6]])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([0.28318205, 0.41703075], dtype=float32)

# Bert Score

In [ ]:
!pip install bert_score

     |████████████████████████████████| 59 kB 2.4 MB/s 


In [ ]:
from bert_score import score


In [ ]:

help(score)

Help on function score in module bert_score.score:

score(cands, refs, model_type=None, num_layers=None, verbose=False, idf=False, device=None, batch_size=64, nthreads=4, all_layers=False, lang=None, return_hash=False, rescale_with_baseline=False, baseline_path=None, use_fast_tokenizer=False)
    BERTScore metric.
    
    Args:
        - :param: `cands` (list of str): candidate sentences
        - :param: `refs` (list of str or list of list of str): reference sentences
        - :param: `model_type` (str): bert specification, default using the suggested
                  model for the target langauge; has to specify at least one of
                  `model_type` or `lang`
        - :param: `num_layers` (int): the layer of representation to use.
                  default using the number of layer tuned on WMT16 correlation data
        - :param: `verbose` (bool): turn on intermediate status update
        - :param: `idf` (bool or dict): use idf weighting, can also be a precomputed idf_

In [ ]:

data

,answer1,answer2,label
0,rotiert,schwingt bei vertikaler Polarisation der elektrische Feldvektor zwischen obe...,0.0
1,in Blautönen gehaltenen Außenwände,Blaues,1.0
2,'Anorthotiko Komma Ergazomenou Laou'',Fortschrittspartei des werktätigen Volkes,2.0
3,aus dem englischen,"von ‚Geißel‘, ‚Seuche‘ und lat. ‚töten‘",0.0
4,Ramadan-Monat,sind üblicherweise an den Rändern der Koranexemplare markiert,0.0
...,...,...,...
420,die Landesverteidigung oder die Außenpolitik,nur bestimmte Angelegenheiten – etwa die Landesverteidigung oder die Außenpo...,1.0
421,"kein Zentralstaat, sondern eine Konföderation von Provinzen",Die Republik war kein Zentralstaat,1.0
422,Holland,Wichtigste Provinz der Union war die Provinz Holland,1.0
423,"Chopin wurde in Żelazowa Wola geboren, einem Dorf in der Gemeinde Brochów, D...",in Żelazowa Wola,1.0


## Vanilla BERTScore

In [ ]:
series_bert_score = []
for i in range(len(squad)):
    _, _, bertscore = score(
        [squad.answer1[i]], 
        [squad.answer2[i]], 
        model_type='bert-base-uncased', 
        num_layers=2)
    series_bert_score.append(bertscore)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
len(series_bert_score)

In [ ]:
squad_bs = squad.copy()
squad_bs['bert_score'] = [score.item() for score in series_bert_score]

squad_bs.sort_values('bert_score').to_csv('squad_bert_score.csv') 
files.download('squad_bert_score.csv')

In [ ]:
german_series_bert_score = []
for i in range(len(german_quad)):
    _, _, bertscore = score(
        [str(german_quad.answer1[i])], 
        [str(german_quad.answer2[i])], 
        model_type='deepset/gelectra-base', 
        num_layers=2)
    german_series_bert_score.append(bertscore)

In [ ]:
german_quad_bert = german_quad.copy()
german_quad_bert['bert_score'] = [score.item() for score in german_series_bert_score]

german_quad_bert.sort_values('bert_score').to_csv('german_quad_bert.csv') 
files.download('german_quad_bert.csv')

In [ ]:

nq_open_sas

In [ ]:
nq_open_series_bert_score = []
for i in range(len(nq_open)):
    _, _, bertscore = score(
        [str(nq_open.answer1[i])], 
        [str(nq_open.answer2[i])], 
        model_type='bert-base-uncased', 
        num_layers=2)
    nq_open_series_bert_score.append(bertscore.item())
nq_open_bert = nq_open_sas.copy()
nq_open_bert['bert_score'] = nq_open_series_bert_score

nq_open_bert.sort_values('bert_score').to_csv('nq_open_bert.csv') 
files.download('nq_open_bert.csv')

## BERTScore Trained 

In [ ]:
nq_open_bert_trained = []
for i in range(len(nq_open)):
    _, _, bertscore = score(
        [str(nq_open.answer1[i])], 
        [str(nq_open.answer2[i])], 
        model_type='bert-base-uncased')
    nq_open_bert_trained.append(bertscore.item())
nq_open_bert_trained_df = nq_open_bert.copy()
nq_open_bert_trained_df['bert_score_prime'] = nq_open_series_bert_score

nq_open_bert_trained_df.sort_values('bert_score_prime').to_csv('nq_open_bert_trained.csv') 
files.download('nq_open_bert_trained.csv')

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
from datasets import load_metric
metric = load_metric("bertscore")

In [ ]:
score()

In [ ]:
metric.compute(
        predictions=[str(german_quad.answer1[0])], 
        references=[str(german_quad.answer2[0])], 
        model_type='T-Systems-onsite/cross-en-de-roberta-sentence-transformer')
        # model_type='deepset/gelectra-base')


KeyError: ignored

In [ ]:
# german_quad_bert_trained = []
# for i in range(len(german_quad)):
#     _, _, bertscore = score(
#         [str(german_quad.answer1[i])], 
#         [str(german_quad.answer2[i])], 
#         # model_type='T-Systems-onsite/cross-en-de-roberta-sentence-transformer')
#         # model_type='deepset/gelectra-base')
#         lang='de', 
#         verbose=False)
#     german_quad_bert_trained.append(bertscore.item())
german_quad_bert_trained_df = german_quad.copy()
german_quad_bert_trained_df['bert_score_prime'] = german_quad_bert_trained

german_quad_bert_trained_df.sort_values('bert_score_prime').to_csv('german_quad_bert_trained.csv') 
files.download('german_quad_bert_trained.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>